In [3]:
import numpy as np
import copy 

# Introduction
This Jupyter Notebook will provide a top down explanation of each function that I created in order to perform Guass elimination and back substitution. Please reference my paper for a bottom up approach to why this procedure works.

Written by Tommy Moawad, tjm165

# 1 Before we start. We need to define some fundemental functions

## 1.1 Row Operations
Row operations were one of the first things we learned in the class and is fundemental to this project.

This function enables us to perform any operation of the form
    r_i = c * r_i - k * r_j
where c and k are constains

For example, suppose we have

[[1, 1, 1],

 [2, 2, 2],
 
 [3, 3, 3]]
 
We can say r_1 = 100 * r_0 - 2 * r_1 such that we now have

[96, 96, 96],

 [2, 2, 2],
 
 [3, 3, 3]

In [4]:
"""
General function to perform a row opperations of the form: r_i = c * r_i - k * r_j

Args:
    matrix: numpy matrix of dimensions m rows by n columns
    c: The coefficient to multiply r_i by (optional by defualt is 1)
    r_i: The index of the row we are applying the operation to
    k: The coefficient to multiply r_j by (optional by defualt is 0)
    r_j: The index of the row we are adding to r_i (optional by defualt is 0)
    
Returns:
    The matrix with the applied row operation such that: r_i = c * r_i - k * r_j
"""
def row_operation(matrix, r_i, c = 1, r_j = 0, k = 0):
    matrix2 = copy.deepcopy(matrix)
    
    matrix2[r_i] = c * matrix2[r_i] - k * matrix2[r_j]
    return matrix2

### Example of Row Operation

In [5]:
matrix = np.matrix([[1, 1, 1], 
                    [2, 2, 2], 
                    [3, 3, 3]])

row_operation(matrix, 0, 100, 1, 2)

matrix([[96, 96, 96],
        [ 2,  2,  2],
        [ 3,  3,  3]])

## 1.2 Row Swap
Another basic operation we learned is row swapping. This is just as essential to Guass Elimination as Row Operation is. 

In [6]:
"""
General function to perform a row swaps

Args:
    matrix: numpy matrix of dimensions m rows by n columns
    r_i: The index of the row to swap with r_j
    r_j: The index of the row to swap with r_i
    
Returns:
    The matrix with the corresponding row swap
"""
def row_swap(matrix, r_i, r_j):
    matrix2 = copy.deepcopy(matrix)
    matrix2[[r_i, r_j]] = matrix[[r_j, r_i]]
    return matrix2

### Example Row Swap

In [7]:
matrix = np.array([[4,3,1], 
                   [5,7,0], 
                   [9,9,3]])
row_swap(matrix, 0, 2)

array([[9, 9, 3],
       [5, 7, 0],
       [4, 3, 1]])

## 1.3 Get Smaller Dimension
Another basic function we need is a way to get the smaller dimension. This is just an if-else statement but it will be useful later on.

Given a matrix with dimensions m by n, if m is smaller then return m, else return n

In [8]:
"""
Obtain the smaller dimension of the matrix. Either 

Args:
    matrix: numpy matrix of dimensions m rows by n columns
    
Returns:
    The smaller dimension of the matrix
"""
def get_smaller_dimension(matrix):
    if len(matrix) < len(matrix.T):
        return len(matrix)
    else:
        return len(matrix.T)

### Examples of Getting the Smaller Dimension

In [9]:
# 2 by 3 matrix. m is smaller
matrix = np.matrix([[1, 1, 1], [2, 2, 2]])
get_smaller_dimension(matrix)

2

In [10]:
# 4 by 3 matrix. n is smaller
matrix = np.matrix([[1, 1, 1], 
                    [2, 2, 2], 
                    [3, 3, 3], 
                    [4, 4, 4]])
get_smaller_dimension(matrix)

3

# 2: Guass Elimination and Back Substitution

There are four parts to Guass Elimination and Back Substitution:
        1. Converted every element to floats. This was simply to avoid technical issues with integer rounding.
        2. Augment A and b into [A|b]
        3. Convert our augmented matrix into row echelon form. This provides us with an upper triangular matrix
        4. Now that we have an upper triangular matrix, we can easily perform back substitution.
        
After following these steps we can have the following function (although we still need to implement some of them!)

In [11]:
"""
Solves the equation Ax = b

Args:
    A: numpy matrix of dimensions m rows by n columns
    b: numpy matrix of dimensions n rows by 1 column
    
Returns:
    The x matrix such that Ax = b
"""
def guass_elimination_and_back_substitution(A, b):
    # First, convert to floats. This helps avoid integer rounding
    A = A.astype(float)
    b = b.astype(float)
    
    augmented = np.hstack((A, b)) # Convert from A, b to [A|b]    
    U = ref(augmented)
    U, b = np.hsplit(U, [np.size(U, 1) - 1]) # Convert from [U|b] to U, b
    x = back_substitution(U, b)
    
    return x

Examples of Guass Elimination and Back Substitution will be provided after walking through all the necessary functions

# 3: Row Echelon Form

The main reason we use row echelon form is because it gives us an upper triangular matrix. This upper triangular matrix makes our life so much easier when performing back substitution. 

There are three steps to row echelon form:
    1. clone a copy of our original matrix. This helps us avoid any technical issues when if we decide to use this matrix elsewhere throughout the project
    2. Convert the matrix into upper triangular form
    3. Make all the diagonals equal to 1
    
After following these steps we can have the following function (although we still need to implement some of them!)

In [12]:
"""
Find the row echelon form of the given matrix
"""
def ref(matrix):
    clone = copy.deepcopy(matrix)
    U = upper_triangular(clone)
    #return make_diagonals_1(U)
    return U

## 3.1 Upper Triangular Matrix

An upper triangular matrix is a matrix such that all elements below the diagonal are 0. This may seem intimidating but don't worry! Just take it column by column. For any column, we need a function that will make all elements zero underneath a specied element.

In [13]:
"""
Convert a matrix into it's upper triangular form
"""
def upper_triangular(matrix):
    matrix2 = copy.deepcopy(matrix)
    
    for i in range(len(matrix.T)): # for each column
        matrix2 = zeros_underneath(matrix2, i, i) #make everything 0 underneath the diagonal
    return matrix2

## 3.2 Making Zeros Under the Diagonal

In 3.1 we said that we need to make all the elements underneath the diagonal to be 0. In this section we will explain how that is actually done. There are two functions for this to be done

    1. We can use a row operation
    2. We can use a row swap 
    
Each function only works under specific conditions. Hence, at every iteration we will check the conditions of our matrix to determine which function to use.

In [58]:
"""
Creates a matrix such that all elements within the same column as a selected index is zero

Args:
    matrix: numpy matrix of dimensions m rows by n columns
    row_index: The row index of the selected element
    col_index: The column index of the selected element.

Returns:
    A new array such that elements from matrix.getitem(row_index + 1, col_index) to matrix.getitem(m, col_index) = 0
"""
def zeros_underneath(matrix, row_index, col_index):
    matrix2 = copy.deepcopy(matrix)
    row_j = row_index #use the row above
    
    for row_i in range(row_index + 1, len(matrix), 1): # for each row underneath row_i
        if matrix2.item(row_j, col_index) == 0:
            matrix2 = row_swap(matrix2, row_i, row_j)
        else:
            k = matrix.item(row_i, col_index) / matrix2.item(row_j, col_index)
            matrix2 = row_operation(matrix2, row_i, 1, row_j, k)
    return matrix2

### Example of Setting Zeros Underneath

In [15]:
matrix = np.matrix([[5], 
                    [6], 
                    [7]])
zeros_underneath(matrix, 0, 0)

[[5]
 [6]
 [7]]



matrix([[5],
        [0],
        [0]])

### Example of Upper Triangular Matrix
This example was found on [YouTube](https://www.youtube.com/watch?v=f-zQJtkgcOE)

In [16]:
matrix = np.matrix([[2, 4, -2], 
                    [4, -2, 6], 
                    [6, -4, 2]])
upper_triangular(matrix)

[[ 2  4 -2]
 [ 4 -2  6]
 [ 6 -4  2]]

[[  2   4  -2]
 [  0 -10  10]
 [  0 -16   8]]

[[  2   4  -2]
 [  0 -10  10]
 [  0   0  -8]]



matrix([[  2,   4,  -2],
        [  0, -10,  10],
        [  0,   0,  -8]])

## 3.3 Make Diagonals 1
The final step to reduced row echelon form is to make the diagonals equal to 1. Recall that up till now, we only have an upper triangular matrix. This is not necessary.

In [17]:
"""
Obtain an upper triangular matrix such that the diagonals are all 1

Args:
    U: numpy matrix upper triangular matrix
    
Returns:
     An upper triangular matrix such that the diagonals are all 1
"""
def make_diagonals_1(U):
    U = copy.deepcopy(U)
    
    for i in range(get_smaller_dimension(U)):
        coef = 1 / U.item(i, i)
        #U[i] = U[i] * coef
        U = row_operation(U, i, coef, 0, 0)
    return U

# 4 Back Substitution
In class, back substitution was always my favorite part. I knew that if I got to this point that I would be able to solve the problem. The basics of back substitution is to 
    1. Start from the bottom of the matrix and work your way up
    2. Within each iteration you are solving one dot product with one unknown.
    3. Hence, the dot product can be expressed as...
        row_i • x_i = b_i
    4. We can then expand the dot product to...
        k * x_i + c = b_i
            where k is the element of row_i that gets dotted with x_i
            and c is the product of all the sums in the dot
    5. Hence with one equation and one unknown we can rearrange to solve for the unknown
        x_i = (b_i - c) / row_i

In [78]:
"""
Solves a system of equations of the form Ux=b

Args:
    U: numpy matrix upper triangular matrix of dimensions m rows by n columns
    b: numpy matrix of dimensions n rows by 1 column
    
Returns:
    The x matrix such that Ux = b
"""
def back_substitution(U, b):
    x = [0] * len(U)
    is_free_variable_found = False
    
    for i in range(len(x) - 1, -1, -1): # substitute from the bottom of the matrix up
        row = U[i].flatten()
        constant = 0
        for j in range(len(x)): # perform the dot products with constants
            if (is_free_variable_found):
                constant = "(" + str(constant) + " + " + str(row.item(j)) + " * " + str(x[j]) + ")"
                print(constant)
            else:
                constant = constant + row.item(j) * x[j]
                print("(" + str(constant) + " + " + str(row.item(j)) + " * " + str(x[j]) + ") = " + str(constant))
        if (not is_free_variable_found):
            if (row.item(i) == 0): #there is a free variable. This dot product is the constant
                if (constant == b.item(i)):
                    is_free_variable_found = True
                    x[i] = "?"
                else:
                    return "There is no solution"
            else:
                x[i] = (b[i] - constant) / row.item(i) # Solve for the unknown
        else:
            x[i] = constant
        
    return x

## Example of Back Substitution

# Examples of Guass Elimination and Back Substitution

In [77]:
A = np.matrix([[2, 1, -1], 
               [3, 2, 1], 
               [2, -1, 2]])
b = np.matrix([[1], 
               [10], 
               [6]])

# Solution is [1, 2, 3]

guass_elimination_and_back_substitution(A, b)

0.0
0.0
0.0
0.0
0.0
[[7.5]]
0.0
[[2.]]
[[-1.]]


[matrix([[1.]]), matrix([[2.]]), matrix([[3.]])]

## a.

In [79]:
A = np.matrix([[1, -1, 2, -1], 
               [2, -2, 3, -3], 
               [1, 1, 1, 0],
               [1, -1, 4, 3]])

b = np.matrix([[-8], 
               [-20], 
               [-2],
               [4]])

# Solution is [-7, 3, 2, 2]

guass_elimination_and_back_substitution(A, b)

(0.0 + 0.0 * 0) = 0.0
(0.0 + 0.0 * 0) = 0.0
(0.0 + 0.0 * 0) = 0.0
(0.0 + 2.0 * 0) = 0.0
(0.0 + 0.0 * 0) = 0.0
(0.0 + 0.0 * 0) = 0.0
(0.0 + -1.0 * 0) = 0.0
([[-2.]] + -1.0 * [[2.]]) = [[-2.]]
(0.0 + 0.0 * 0) = 0.0
(0.0 + 2.0 * 0) = 0.0
([[-2.]] + -1.0 * [[2.]]) = [[-2.]]
([[0.]] + 1.0 * [[2.]]) = [[0.]]
(0.0 + 1.0 * 0) = 0.0
([[-3.]] + -1.0 * [[3.]]) = [[-3.]]
([[1.]] + 2.0 * [[2.]]) = [[1.]]
([[-1.]] + -1.0 * [[2.]]) = [[-1.]]


[matrix([[-7.]]), matrix([[3.]]), matrix([[2.]]), matrix([[2.]])]

## b.

In [69]:
A = np.matrix([[1, 1, 1], 
               [2, 2, 1], 
               [1, 1, 2]])

b = np.matrix([[4], 
               [6], 
               [6]])

guass_elimination_and_back_substitution(A, b)

(0 + 1.0 * 0)
((0 + 1.0 * 0) + 1.0 * ?)
(((0 + 1.0 * 0) + 1.0 * ?) + 1.0 * [[2.]])


['(((0 + 1.0 * 0) + 1.0 * ?) + 1.0 * [[2.]])', '?', matrix([[2.]])]

## c.

In [70]:
A = np.matrix([[1, 1, 1], 
               [2, 2, 1], 
               [1, 1, 2]])

b = np.matrix([[4], 
               [4], 
               [6]])

guass_elimination_and_back_substitution(A, b)

'There is no solution'

In [74]:
'(((0 + 1.0 * 0) + 1.0 * ?) + 1.0 * [[2.]])'.replace('?', '(-1)')

'(((0 + 1.0 * 0) + 1.0 * (-1)) + 1.0 * [[2.]])'

In [75]:
eval('(((0 + 1.0 * 0) + (-1) * (1)) + 1.0 * 2.)')

1.0